## FindMedianEmbeddings_ForEachOf7Clusters
This script:
- Reads the expanded dataframe (one row per image) with cluster assignments (output of script 5)
- For a chosen k, computes per-LSOA summary statistics:
    - Count and percentage of images in each cluster
    - Mean, max, and median embedding within each cluster
    - Overall mean, max, and median embedding (across all clusters)
- Saves a single pickle file combining all of the above

In [1]:
import os
import numpy as np
import pandas as pd
from functools import reduce

In [2]:
from directory_filepaths import *

In [3]:
k = 4

### Get data

In [4]:
# Load expanded dataframe with cluster assignments (output of script 5)
expanded_gdf = pd.read_pickle(os.path.join(data_dir, "one_row_per_image_cleaned_with_cluster_numbers.pkl"))
print(f"Loaded {len(expanded_gdf)} image rows")

Loaded 75586 image rows


# Create a dataframe with % of images in each category, in each LSOA 

In [5]:
category_column = f"scene_cluster_{k}"

# Count images per (LSOA, cluster)
category_counts = (
    expanded_gdf.groupby(["LSOA21CD", category_column])
    .size()
    .reset_index(name="count")
)

# Total images per LSOA
total_counts = expanded_gdf.groupby("LSOA21CD").size().reset_index(name="total_images")

# Merge and compute percentages
category_counts = category_counts.merge(total_counts, on="LSOA21CD")
category_counts["pct"] = category_counts["count"] / category_counts["total_images"] * 100

# Pivot to wide format: one column per cluster for counts and percentages
counts_wide = (
    category_counts.pivot(index="LSOA21CD", columns=category_column, values="count")
    .fillna(0)
    .add_prefix("count_")
)
pct_wide = (
    category_counts.pivot(index="LSOA21CD", columns=category_column, values="pct")
    .fillna(0)
    .add_prefix("pct_")
)

# Combine counts, percentages, and totals per LSOA
lsoa_summary = total_counts.set_index("LSOA21CD").join([counts_wide, pct_wide])

# Sanity check - do the counts sum to total_images?
# Compute sums into local variables instead of creating temporary DataFrame columns
counts_sum = lsoa_summary.filter(like="count_").sum(axis=1)
assert np.all(counts_sum == lsoa_summary["total_images"]), "Counts do not sum to total_images for some LSOAs"
# And do the percentages sum to 100%?
pct_sum = lsoa_summary.filter(like="pct_").sum(axis=1)
assert np.allclose(pct_sum, 100), "Percentages do not sum to 100% for some LSOAs"

lsoa_summary.head()

,total_images,count_1,count_2,count_3,count_4,pct_1,pct_2,pct_3,pct_4
LSOA21CD,,,,,,,,,
E01004766,64,14.0,25.0,5.0,20.0,21.875000,39.062500,7.812500,31.250000
E01004767,72,19.0,20.0,8.0,25.0,26.388889,27.777778,11.111111,34.722222
E01004768,44,3.0,15.0,15.0,11.0,6.818182,34.090909,34.090909,25.000000
E01004769,40,3.0,13.0,17.0,7.0,7.500000,32.500000,42.500000,17.500000
E01004770,40,4.0,18.0,9.0,9.0,10.000000,45.000000,22.500000,22.500000


# Find mean/median/max embedding in each LSOA, also by category

In [6]:
def mean_embed(series):
    return np.mean(np.stack(series.values), axis=0)

def max_embed(series):
    return np.max(np.stack(series.values), axis=0)

def median_embed(series):
    return np.median(np.stack(series.values), axis=0)

agg_funcs = {"mean": mean_embed, "max": max_embed, "median": median_embed}
categories = sorted(expanded_gdf[category_column].unique())

all_dfs = []

for agg_name, func in agg_funcs.items():
    dfs = []

    # Per-cluster embedding aggregation
    for cat in categories:
        emb_cat = (
            expanded_gdf[expanded_gdf[category_column] == cat]
            .groupby("LSOA21CD")["embedding"]
            .apply(func)
            .reset_index()
            .rename(columns={"embedding": f"{cat}_{agg_name}"})
        )
        dfs.append(emb_cat)

    # Merge all clusters, then add overall (all images in LSOA)
    merged = reduce(lambda left, right: pd.merge(left, right, on="LSOA21CD", how="outer"), dfs)

    overall = (
        expanded_gdf.groupby("LSOA21CD")["embedding"]
        .apply(func)
        .reset_index()
        .rename(columns={"embedding": f"overall_{agg_name}"})
    )
    merged = merged.merge(overall, on="LSOA21CD", how="left")
    all_dfs.append(merged)

# Combine mean, max, and median into a single DataFrame
final_df = reduce(lambda left, right: pd.merge(left, right, on="LSOA21CD", how="outer"), all_dfs)
print(f"{len(final_df)} LSOAs, {len(final_df.columns) - 1} feature columns")

1695 LSOAs, 15 feature columns


In [7]:
final_df

,LSOA21CD,1_mean,2_mean,3_mean,4_mean,overall_mean,1_max,2_max,3_max,4_max,overall_max,1_median,2_median,3_median,4_median,overall_median
0,E01004766,"[0.022396863, -0.060344696, 0.0025673595, -0.0...","[0.04366791, -0.061499022, -0.007403755, -0.00...","[0.028472137, -0.056408692, -0.009085846, -0.0...","[0.03406372, -0.054208376, 0.015108442, -0.007...","[0.03482639, -0.058570504, 0.0016810745, -0.00...","[0.05166626, -0.014503479, 0.020477295, 0.0179...","[0.06555176, -0.036071777, 0.027816772, 0.0227...","[0.06604004, -0.025939941, 0.011779785, 0.0054...","[0.06878662, 0.02444458, 0.04058838, 0.0209045...","[0.06878662, 0.02444458, 0.04058838, 0.0227813...","[0.027572632, -0.06323242, 0.004722595, -0.008...","[0.046936035, -0.057922363, -0.009925842, -0.0...","[0.01940918, -0.055511475, -0.015129089, 4.172...","[0.029373169, -0.060531616, 0.015346527, -0.00...","[0.037399292, -0.060531616, 0.0037984848, -0.0..."
1,E01004767,"[0.025314808, -0.060883623, -0.0066822954, -0....","[0.041760635, -0.066438295, -0.006786442, -0.0...","[0.022837639, -0.06278992, -0.020924568, 0.008...","[0.030664215, -0.05010452, 0.013643112, -0.008...","[0.031465285, -0.058895655, -0.0012362666, -0....","[0.061798096, 0.0012311935, 0.020095825, 0.041...","[0.07910156, -0.040283203, 0.022415161, 0.0113...","[0.05206299, -0.046447754, -0.003540039, 0.024...","[0.0647583, 0.007045746, 0.057800293, 0.018951...","[0.07910156, 0.007045746, 0.057800293, 0.04144...","[0.02519226, -0.061187744, -0.005306244, -0.00...","[0.04458618, -0.06074524, -0.009237289, -0.010...","[0.024368286, -0.0566864, -0.016716003, 0.0066...","[0.026123047, -0.047607422, 0.012107849, -0.01...","[0.028137207, -0.05706787, -0.002474308, -0.00..."
2,E01004768,"[0.033966064, -0.067352295, -0.0005553563, -0....","[0.0387441, -0.065450035, -0.0010991414, -0.01...","[0.050431825, -0.06296793, -0.013619105, 0.008...","[0.029031234, -0.059611928, 0.018901478, -0.01...","[0.03997456, -0.06327403, -0.0003300797, -0.00...","[0.039978027, -0.029022217, 0.009536743, -0.00...","[0.059661865, -0.04296875, 0.023986816, 0.0077...","[0.08831787, -0.042388916, 0.014816284, 0.0340...","[0.066711426, -0.04043579, 0.035186768, 0.0102...","[0.08831787, -0.029022217, 0.035186768, 0.0340...","[0.034729004, -0.072265625, 0.0019273758, -0.0...","[0.036499023, -0.06744385, -0.0031776428, -0.0...","[0.048187256, -0.055847168, -0.014961243, 0.00...","[0.026473999, -0.060943604, 0.022583008, -0.01...","[0.036331177, -0.06199646, -0.0030584335, -0.0..."
3,E01004769,"[0.033533733, -0.05350749, -0.012964885, -0.00...","[0.031613275, -0.06352821, -0.007081692, -0.00...","[0.042486753, -0.05386173, -0.02130284, -0.003...","[0.04457528, -0.07030378, 0.012783595, -0.0155...","[0.038646888, -0.059854127, -0.0100904945, -0....","[0.044647217, -0.045562744, -0.009788513, 0.00...","[0.059448242, -0.03744507, 0.024093628, 0.0186...","[0.059020996, -0.026062012, 4.9591064e-05, 0.0...","[0.07623291, -0.04095459, 0.017303467, -0.0046...","[0.07623291, -0.026062012, 0.024093628, 0.0186...","[0.0446167, -0.04586792, -0.013145447, -0.0044...","[0.03062439, -0.06689453, -0.0074157715, -0.00...","[0.04067993, -0.053344727, -0.023239136, -0.00...","[0.046875, -0.07550049, 0.012901306, -0.015823...","[0.039871216, -0.058303833, -0.011016846, -0.0..."
4,E01004770,"[0.025318146, -0.04863739, -0.00014591217, -0....","[0.040929157, -0.055555556, -0.011239065, -0.0...","[0.02157296, -0.061513264, -0.014335632, 0.006...","[0.03234058, -0.06099786, 0.0149474675, -0.010...","[0.03308048, -0.05742874, -0.0049345074, -0.00...","[0.040405273, -0.021728516, 0.014060974, 0.006...","[0.06982422, -0.035095215, 0.022827148, 0.0055...","[0.04348755, -0.034942627, -0.0025405884, 0.02...","[0.045410156, -0.03326416, 0.039764404, 0.0095...","[0.06982422, -0.021728516, 0.039764404, 0.0282...","[0.023727417, -0.053451538, -0.00116539, -0.00...","[0.038406372, -0.053024292, -0.01209259, -0.01...","[0.022415161, -0.062316895, -0.005367279, 0.00...","[0.03

### Save

In [8]:
# Attach the LSOA count/percentage summary and save
final_df = final_df.merge(lsoa_summary, on="LSOA21CD")

output_path = os.path.join(data_dir, "per_lsoa_embedding_summaries", "median_embedding_per_cluster.pkl")
os.makedirs(os.path.dirname(output_path), exist_ok=True)
final_df.to_pickle(output_path)
print(f"Saved to {output_path}")

Saved to ../../../../data/embeddings/per_lsoa_embedding_summaries/median_embedding_per_cluster.pkl


In [9]:
print(final_df['count_1'].sum())
print(final_df['count_2'].sum())

14472.0
27936.0
